In [ ]:
import os

import seaborn as sns

from ray import init, rllib, tune, shutdown

In [ ]:
from attack_simulator.agents import ATTACKERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [ ]:
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
attackers = list(ATTACKERS)
attackers

In [ ]:
# Two blank lines added to please `flakehell lint`


class NoAction(rllib.policy.Policy):
    def compute_actions(self, observations, *args, **kwargs):
        return [0 for _ in observations], [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass

    def learn_on_batch(self, samples):
        return {}


no_action = rllib.agents.trainer_template.build_trainer(name='NoAction', default_policy=NoAction)

In [ ]:
if os.path.isdir('/var/run/secrets/kubernetes.io'):  # inside k8s pod
    init(address='auto')
else:
    # listen on all interfaces inside a container for port-forwarding to work
    dashboard_host = '0.0.0.0' if os.path.exists("/.dockerenv") else '127.0.0.1'
    init(num_cpus=4, dashboard_host=dashboard_host)
    
# ALTERNATIVE: use the "Ray client" to connect to a remote cluster
# Unfortunately, JupyterNotebookReporter displays an object reference
# <IPython.core.display.HTML object> instead of content...
# --- --- ---
#
# from ray.util.client import worker
#
# worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1
#
# ray_client_server = 'host.docker.internal' if os.path.exists("/.dockerenv") else '127.0.0.1'
# init(address=f'ray://{ray_client_server}:10001')

In [ ]:
config = dict(
    framework='torch',
    env=AttackSimulationEnv,
    env_config=dict(attack_graph=tune.grid_search(graphs), attacker=tune.grid_search(attackers)),
    seed=tune.grid_search([0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]),
    simple_optimizer=True,  # not really training, no need for the MultiGPU version
    num_workers=8,
    train_batch_size=1,
    rollout_fragment_length=1,
    batch_mode='complete_episodes',
    gamma=1.0,
)

num_episodes = 5  # run 5 episodes on the same environment

In [ ]:
results = tune.run(
    no_action,
    config=config,
    stop=dict(training_iteration=num_episodes),
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
)

In [ ]:
shutdown()

In [ ]:
rename = {
    'config.env_config.attack_graph': 'graph',
    'config.env_config.attacker': 'Attacker',
    'hist_stats.episode_reward': 'returns',
    'hist_stats.episode_lengths': 'lengths',
}

df = results.results_df[rename.keys()].rename(columns=rename)
df['Graph size'] = df['graph'].apply(lambda g: g.num_attacks)
del df['graph']
df['tuple'] = df.apply(lambda t: list(zip(t.returns, t.lengths)), axis='columns')
del df['returns']
del df['lengths']
df = df.explode('tuple', ignore_index=True)
df[['Returns', 'Episode lengths']] = df['tuple'].tolist()
del df['tuple']
df

In [ ]:
sns.set(style='darkgrid', rc={'figure.figsize': (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x='Graph size', y='Returns', hue='Attacker', ci='sd')
g.legend(title='Attacker', loc='upper right')
g.set_title('Defender: no-action')

In [ ]:
g = sns.lineplot(data=df, x='Graph size', y='Episode lengths', hue='Attacker', ci='sd')
g.legend(title='Attacker', loc='upper left')
g.set_title('Defender: no-action')

In [ ]:
df.groupby('Attacker').describe()